In [1]:

import sys
import pandas as pd

In [2]:
import numpy as np
import psycopg2
from sqlalchemy import create_engine

In [4]:
biz_info_df = pd.read_csv('ontario_biz_data.csv')
biz_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33412 entries, 0 to 33411
Data columns (total 67 columns):
Unnamed: 0                               33412 non-null int64
address                                  33012 non-null object
attributes.AcceptsInsurance              566 non-null object
attributes.AgesAllowed                   16 non-null object
attributes.Alcohol                       11208 non-null object
attributes.Ambience                      11561 non-null object
attributes.BYOB                          0 non-null float64
attributes.BYOBCorkage                   1 non-null object
attributes.BestNights                    533 non-null object
attributes.BikeParking                   17313 non-null object
attributes.BusinessAcceptsBitcoin        2 non-null object
attributes.BusinessAcceptsCreditCards    613 non-null object
attributes.BusinessParking               21523 non-null object
attributes.ByAppointmentOnly             5050 non-null object
attributes.Caters              

/Users/my_macbook/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,10,28,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
biz_info_df.drop(['Unnamed: 0'],inplace=True, axis=1)

In [6]:
biz_info_df.count()

address                                  33012
attributes.AcceptsInsurance                566
attributes.AgesAllowed                      16
attributes.Alcohol                       11208
attributes.Ambience                      11561
attributes.BYOB                              0
attributes.BYOBCorkage                       1
attributes.BestNights                      533
attributes.BikeParking                   17313
attributes.BusinessAcceptsBitcoin            2
attributes.BusinessAcceptsCreditCards      613
attributes.BusinessParking               21523
attributes.ByAppointmentOnly              5050
attributes.Caters                         9030
attributes.CoatCheck                       648
attributes.Corkage                           0
attributes.DietaryRestrictions              17
attributes.DogsAllowed                    1590
attributes.DriveThru                       552
attributes.GoodForDancing                  855
attributes.GoodForKids                   14726
attributes.Go

In [8]:
biz_info_df.head()

,address,attributes.AcceptsInsurance,attributes.AgesAllowed,attributes.Alcohol,attributes.Ambience,attributes.BYOB,attributes.BYOBCorkage,attributes.BestNights,attributes.BikeParking,attributes.BusinessAcceptsBitcoin,...,stars,state,checkin_date,user_id,tip_text,tip_compliment_count,tip_date,photo_id,label,caption
0,30 Eglinton Avenue W,NaN,NaN,u'full_bar',"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,False,NaN,...,2.5,ON,"2010-07-05 16:56:31, 2010-12-21 20:30:01, 2010...","fENhF1F6I9c2ssnkfprYzw,RscJR5zqb3WL4eQezBX5rA,...","Spring & egg rolls are great, get them with ex...",1.0,"2016-08-08 01:48:14,2013-06-21 23:44:22,2017-1...","2LEm7R6KYy001sUrXvsnKA,bJnzJWtyXc9Acl4ev7KCXA,...","outside,food,inside","null,??, ???, ??,null"
1,"Credit Valley Town Plaza, F2 - 6045 Creditview Rd",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.5,ON,"2014-03-31 21:46:17, 2015-01-27 19:05:03, 2018...",jnB_saJqNfOmVoCWquhAzg,Good service,0.0,2014-03-31 21:46:39,NaN,NaN,NaN
2,1455 16th Avenue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,...,4.0,ON,"2016-04-19 17:38:29, 2016-04-22 01:14:54, 2016...","CxDOIDnH8gp9KXzpBHJYXw,5vRBvQEbR12jXzC50cBYqw","Replacing My Yoga Studio,Went there for the fi...",0.0,"2016-04-19 16:16:40,2017-09-20 15:10:00",NaN,NaN,NaN
3,"445 King Street W, Suite 101",True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,ON,"2010-10-23 14:00:26, 2010-10-24 16:33:33, 2010...","n1bfatw5wT1eRYWmHSLo9g,gLSQCEhGy7-uRnNbNnVykg","One-week guest pass is $44.,Very average. Best...",0.0,"2014-08-07 03:35:10,2013-07-28 21:31:25",NaN,NaN,NaN
4,"865 York Mills Road, Unit 1",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,...,4.5,ON,"2016-01-24 00:15:08, 2016-01-25 22:42:18, 2016...","keLUgL_4y60BkppiAsIk8Q,keLUgL_4y60BkppiAsIk8Q",Tried the tiramisu mini cake $16.. very light!...,1.0,"2016-03-22 20:21:20,2016-01-23 20:02:08",NaN,NaN,NaN


In [7]:
col_list = biz_info_df.columns

In [8]:
col_list

Index(['address', 'attributes.AcceptsInsurance', 'attributes.AgesAllowed',
       'attributes.Alcohol', 'attributes.Ambience', 'attributes.BYOB',
       'attributes.BYOBCorkage', 'attributes.BestNights',
       'attributes.BikeParking', 'attributes.BusinessAcceptsBitcoin',
       'attributes.BusinessAcceptsCreditCards', 'attributes.BusinessParking',
       'attributes.ByAppointmentOnly', 'attributes.Caters',
       'attributes.CoatCheck', 'attributes.Corkage',
       'attributes.DietaryRestrictions', 'attributes.DogsAllowed',
       'attributes.DriveThru', 'attributes.GoodForDancing',
       'attributes.GoodForKids', 'attributes.GoodForMeal',
       'attributes.HairSpecializesIn', 'attributes.HappyHour',
       'attributes.HasTV', 'attributes.Music', 'attributes.NoiseLevel',
       'attributes.Open24Hours', 'attributes.OutdoorSeating',
       'attributes.RestaurantsAttire', 'attributes.RestaurantsCounterService',
       'attributes.RestaurantsDelivery', 'attributes.RestaurantsGoodForGr

In [9]:
basic_model_col = ['business_id','latitude', 'longitude','name', 'address','city', 'state','postal_code', 
                  'categories','is_open','review_count', 'stars']

In [10]:
basic_model_df = biz_info_df[basic_model_col]
basic_model_df.head()

,business_id,latitude,longitude,name,address,city,state,postal_code,categories,is_open,review_count,stars
0,QXAEGFB4oINsVuTFxEYKFQ,43.605499,-79.652289,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,"Specialty Food, Restaurants, Dim Sum, Imported...",1,128,2.5
1,68dUKd8_8liJ7in4aWOSEA,43.599475,-79.711584,The UPS Store,"Credit Valley Town Plaza, F2 - 6045 Creditview Rd",Mississauga,ON,L5V 0B1,"Shipping Centers, Couriers & Delivery Services...",1,3,2.5
2,A98xW4qb7vOTguggHFs7Ng,43.861503,-79.388499,Hot Yoga Wellness,1455 16th Avenue,Richmond Hill,ON,L4B 3G6,"Fitness & Instruction, Active Life, Yoga",1,4,4.0
3,zSpQmEBvRe3IhTUlMSA6HQ,43.645041,-79.395799,Totum Life Science,"445 King Street W, Suite 101",Toronto,ON,M5V 1K4,"Trainers, Health & Medical, Active Life, Physi...",1,23,4.0
4,C9oCPomVP0mtKa8z99E3gg,43.754093,-79.349548,Bakery Gateau,"865 York Mills Road, Unit 1",Toronto,ON,M3B 1Y6,"Bakeries, Food",1,8,4.5


In [11]:
basic_model_df.count()

business_id     33412
latitude        33412
longitude       33412
name            33412
address         33012
city            33412
state           33412
postal_code     33265
categories      33342
is_open         33412
review_count    33412
stars           33412
dtype: int64

In [12]:
#basic_model_df.to_csv('basic_model.csv')

In [11]:
basic_model_df.fillna('null')

,business_id,latitude,longitude,name,address,city,state,postal_code,categories,is_open,review_count,stars
0,QXAEGFB4oINsVuTFxEYKFQ,43.605499,-79.652289,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,"Specialty Food, Restaurants, Dim Sum, Imported...",1,128,2.5
1,68dUKd8_8liJ7in4aWOSEA,43.599475,-79.711584,The UPS Store,"Credit Valley Town Plaza, F2 - 6045 Creditview Rd",Mississauga,ON,L5V 0B1,"Shipping Centers, Couriers & Delivery Services...",1,3,2.5
2,A98xW4qb7vOTguggHFs7Ng,43.861503,-79.388499,Hot Yoga Wellness,1455 16th Avenue,Richmond Hill,ON,L4B 3G6,"Fitness & Instruction, Active Life, Yoga",1,4,4.0
3,zSpQmEBvRe3IhTUlMSA6HQ,43.645041,-79.395799,Totum Life Science,"445 King Street W, Suite 101",Toronto,ON,M5V 1K4,"Trainers, Health & Medical, Active Life, Physi...",1,23,4.0
4,C9oCPomVP0mtKa8z99E3gg,43.754093,-79.349548,Bakery Gateau,"865 York Mills Road, Unit 1",Toronto,ON,M3B 1Y6,"Bakeries, Food",1,8,4.5
5,C9keC4mWuXdl2mYFHZXudQ,43.670744,-79.391385,Uncle Otis Clothing,26 Bellair St,Toronto,ON,M5R 2C7,"Fashion, Sporting Goods, Men's Clothing, Shopp...",1,12,5.0
6,NDuUMJfrWk52RA-H-OtrpA,43.642889,-79.425429,Bolt Fresh Bar,1170 Queen Street W,Toronto,ON,M6J 1J5,"Juice Bars & Smoothies, Food, Restaurants, Fas...",1,57,3.0
7,SP_YXIEwkFPPl_9anCYmpQ,43.660494,-79.432099,The Steady Cafe & Bar,1051 Bloor Street W,Toronto,ON,M6H 1M4,"Restaurants, Nightlife, Breakfast & Brunch, Ve...",0,29,3.5
8,mlHC2XcU9Bows6cnYEmRgg,43.655420,-79.413352,Mad Crush Wine Bar,582 College Street,Toronto,ON,M6G 1B3,"Restaurants, Breakfast & Brunch, Bars, Modern ...",0,9,4.0
9,6l00a9Gkxkcp_y-GPjYBiw,44.058765,-79.429115,J's Fish & Chips,"17205 Leslie Street, Unit 2",Newmarket,ON,L3Y 8Y8,"Fish & Chips, Restaurants",1,12,4.5


In [13]:
categories_list = pd.Series(basic_model_df['categories']).tolist()
categories_list

['Specialty Food, Restaurants, Dim Sum, Imported Food, Food, Chinese, Ethnic Food, Seafood',
 'Shipping Centers, Couriers & Delivery Services, Local Services, Printing Services',
 'Fitness & Instruction, Active Life, Yoga',
 'Trainers, Health & Medical, Active Life, Physical Therapy, Gyms, Fitness & Instruction',
 'Bakeries, Food',
 "Fashion, Sporting Goods, Men's Clothing, Shopping, Sports Wear",
 'Juice Bars & Smoothies, Food, Restaurants, Fast Food, Vegan',
 'Restaurants, Nightlife, Breakfast & Brunch, Vegan, Food, Tapas Bars, Bars, Southern, Coffee & Tea',
 'Restaurants, Breakfast & Brunch, Bars, Modern European, Nightlife, Wine Bars',
 'Fish & Chips, Restaurants',
 'Restaurants, Korean',
 'Sandwiches, Desserts, Custom Cakes, Bakeries, Patisserie/Cake Shop, Cafes, Restaurants, Food, Breakfast & Brunch, Coffee & Tea',
 'Italian, Restaurants',
 'Shopping, Discount Store',
 'Beauty & Spas, Nail Salons',
 'Tex-Mex, Mexican, Restaurants',
 'Bakeries, Food, Event Planning & Services, Des

In [14]:
cat = categories_list[0].split(',')
cat

['Specialty Food',
 ' Restaurants',
 ' Dim Sum',
 ' Imported Food',
 ' Food',
 ' Chinese',
 ' Ethnic Food',
 ' Seafood']

In [15]:
cat[1][0]

' '

In [16]:
single_category = []

for i in range(0,len(categories_list)):
    try:
        temp = categories_list[i].split(',')
        for j in range(0,len(temp)):
            if temp[j][0]==' ':
                temp[j] = temp[j][1:]
            single_category.append(temp[j])
    except:
        pass
    

single_category

 

['Specialty Food',
 'Restaurants',
 'Dim Sum',
 'Imported Food',
 'Food',
 'Chinese',
 'Ethnic Food',
 'Seafood',
 'Shipping Centers',
 'Couriers & Delivery Services',
 'Local Services',
 'Printing Services',
 'Fitness & Instruction',
 'Active Life',
 'Yoga',
 'Trainers',
 'Health & Medical',
 'Active Life',
 'Physical Therapy',
 'Gyms',
 'Fitness & Instruction',
 'Bakeries',
 'Food',
 'Fashion',
 'Sporting Goods',
 "Men's Clothing",
 'Shopping',
 'Sports Wear',
 'Juice Bars & Smoothies',
 'Food',
 'Restaurants',
 'Fast Food',
 'Vegan',
 'Restaurants',
 'Nightlife',
 'Breakfast & Brunch',
 'Vegan',
 'Food',
 'Tapas Bars',
 'Bars',
 'Southern',
 'Coffee & Tea',
 'Restaurants',
 'Breakfast & Brunch',
 'Bars',
 'Modern European',
 'Nightlife',
 'Wine Bars',
 'Fish & Chips',
 'Restaurants',
 'Restaurants',
 'Korean',
 'Sandwiches',
 'Desserts',
 'Custom Cakes',
 'Bakeries',
 'Patisserie/Cake Shop',
 'Cafes',
 'Restaurants',
 'Food',
 'Breakfast & Brunch',
 'Coffee & Tea',
 'Italian',
 'Res

In [35]:
single_category_set = set(single_category)
len(single_category)

126923

In [36]:
type(single_category_set)

set

In [37]:
single_category_set

{'& Probates',
 '3D Printing',
 'Acai Bowls',
 'Accessories',
 'Accountants',
 'Acne Treatment',
 'Active Life',
 'Acupuncture',
 'Adult',
 'Adult Education',
 'Adult Entertainment',
 'Advertising',
 'Aerial Fitness',
 'Aestheticians',
 'Afghan',
 'African',
 'Air Duct Cleaning',
 'Airlines',
 'Airport Lounges',
 'Airport Shuttles',
 'Airport Terminals',
 'Airports',
 'Airsoft',
 'Alternative Medicine',
 'Amateur Sports Teams',
 'American (New)',
 'American (Traditional)',
 'Amusement Parks',
 'Animal Shelters',
 'Antiques',
 'Apartments',
 'Appliances',
 'Appliances & Repair',
 'Appraisal Services',
 'Aquarium Services',
 'Aquariums',
 'Arabian',
 'Arcades',
 'Archery',
 'Architects',
 'Argentine',
 'Art Classes',
 'Art Galleries',
 'Art Museums',
 'Art Restoration',
 'Art Schools',
 'Art Supplies',
 'Arts & Crafts',
 'Arts & Entertainment',
 'Asian Fusion',
 'Astrologers',
 'Auction Houses',
 'Audiologist',
 'Australian',
 'Austrian',
 'Auto Customization',
 'Auto Detailing',
 'Auto 

In [38]:
basic_model_df.head()

,business_id,latitude,longitude,name,address,city,state,postal_code,categories,is_open,review_count,stars
0,QXAEGFB4oINsVuTFxEYKFQ,43.605499,-79.652289,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,"Specialty Food, Restaurants, Dim Sum, Imported...",1,128,2.5
1,68dUKd8_8liJ7in4aWOSEA,43.599475,-79.711584,The UPS Store,"Credit Valley Town Plaza, F2 - 6045 Creditview Rd",Mississauga,ON,L5V 0B1,"Shipping Centers, Couriers & Delivery Services...",1,3,2.5
2,A98xW4qb7vOTguggHFs7Ng,43.861503,-79.388499,Hot Yoga Wellness,1455 16th Avenue,Richmond Hill,ON,L4B 3G6,"Fitness & Instruction, Active Life, Yoga",1,4,4.0
3,zSpQmEBvRe3IhTUlMSA6HQ,43.645041,-79.395799,Totum Life Science,"445 King Street W, Suite 101",Toronto,ON,M5V 1K4,"Trainers, Health & Medical, Active Life, Physi...",1,23,4.0
4,C9oCPomVP0mtKa8z99E3gg,43.754093,-79.349548,Bakery Gateau,"865 York Mills Road, Unit 1",Toronto,ON,M3B 1Y6,"Bakeries, Food",1,8,4.5


In [39]:
basic_model_df.columns

Index(['business_id', 'latitude', 'longitude', 'name', 'address', 'city',
       'state', 'postal_code', 'categories', 'is_open', 'review_count',
       'stars'],
      dtype='object')

In [41]:
#Load into the PostgreSQL
engine = create_engine('postgresql+psycopg2://postgres:postgres@@127.0.0.1/yelp_data')


In [42]:
basic_model_df.to_sql('yelp_on_basic_data',  con=engine)